# Generate data

First, we need to determine the episode number to generate. The episode will be saved in at ./data/episode{numer}.

In [1]:
EPISODE = 4

Then, we instate the cameras' information for later use. 

In [2]:
CAMERAS = ["top", "front", "side-1", "side-2"]

Then, we create the folder which holds the episode data.


In [3]:
import os
episode_folder_base = 'data'
episode_folder = os.path.join(episode_folder_base, f'episode{EPISODE}')
if not os.path.exists(episode_folder):
    os.makedirs(episode_folder)
for camera in CAMERAS:
    camera_depth = os.path.join(episode_folder, camera.replace("-", "_")+"_depth")
    camera_rgb = os.path.join(episode_folder, camera.replace("-", "_")+"_rgb")
    if not os.path.exists(camera_depth):
        os.makedirs(camera_depth)
    if not os.path.exists(camera_rgb):
        os.makedirs(camera_rgb)

Next thing is to read from the saves folder to generate the episode. The first step is to copy the current images from the saves folder to the data folder and re-format the names (e.g. 0.png, 1.png, ...).

In [4]:
import shutil

saves_cameras_folder = 'saves/cameras/'
depth_images_path = []
rgb_images_path = []
for camera in CAMERAS:
    camera_folder = os.path.join(saves_cameras_folder, camera.replace("-", "_"))
    images_folder = os.path.join(camera_folder, 'images')
    images = os.listdir(images_folder)
    depth_images = [image for image in images if image.startswith('depth')]
    rgb_images = [image for image in images if image.startswith('rgb')]
    depth_images.sort()
    rgb_images.sort()
    i = 0
    for depth_image, rgb_image in zip(depth_images, rgb_images):
        depth_image_path = os.path.join(images_folder, depth_image)
        rgb_image_path = os.path.join(images_folder, rgb_image)
        depth_image_new_path = os.path.join(episode_folder, camera.replace("-", "_")+"_depth", str(i)+".png")
        rgb_image_new_path = os.path.join(episode_folder, camera.replace("-", "_")+"_rgb", str(i)+".png")
        shutil.copyfile(depth_image_path, depth_image_new_path)
        shutil.copyfile(rgb_image_path, rgb_image_new_path)
        depth_images_path.append(depth_image_new_path)
        rgb_images_path.append(rgb_image_new_path)
        i += 1

Next step is to generate the Observation objects and save them as pickle file in the data folder.

In [6]:
import numpy as np
from rlbench.backend.observation import Observation
import pickle

extrinsics_filename = 'ex_2023_1_29_16_48_45.npy'
intrinsics_filename = 'in_2023_1_29_16_48_45.npy'

camera_far = 2
camera_near = 0.16
misc = {}
for camera in CAMERAS:
    camera_extrinsics = np.load(os.path.join(saves_cameras_folder, camera.replace("-", "_"), 'matrices', extrinsics_filename))
    camera_intrinsics = np.load(os.path.join(saves_cameras_folder, camera.replace("-", "_"), 'matrices', intrinsics_filename))
    misc[camera.replace("-", "_")+"_camera_extrinsics"] = camera_extrinsics
    misc[camera.replace("-", "_")+"_camera_intrinsics"] = camera_intrinsics
    misc[camera.replace("-", "_")+"_camera_far"] = camera_far
    misc[camera.replace("-", "_")+"_camera_near"] = camera_near

obs = []
for depth_image_path, rgb_image_path in zip(depth_images_path, rgb_images_path):
    obs.append(Observation(misc=misc))

with open(f'data/episode{EPISODE}/low_dim_obs.pkl', "wb") as f:
    pickle.dump(obs, f)    

# VISUALIZATION

In [ ]:
from rlbench.backend.utils import extract_obs

import pprint

# what to visualize
episode_idx_to_visualize = 1 # out of 10 demos
ts = 0 # timestep out of total timesteps

# get demo
demo = get_stored_demo(data_path=data_path,
                      index=episode_idx_to_visualize)    

pprint.pprint(demo[ts].front_point_cloud)

# visualize the point cloud
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(demo[ts].front_point_cloud[:, 0], demo[ts].front_point_cloud[:, 1], demo[ts].front_point_cloud[:, 2], c=demo[ts].front_point_cloud[:, 2], cmap='viridis', linewidth=0.5)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# extract obs at timestep                 
obs_dict = extract_obs(demo[ts], CAMERAS, t=ts)

# print(obs_dict)

# total timesteps in demo
print(f"Demo {episode_idx_to_visualize} | {len(demo)} total steps\n")

# plot rgb and depth at timestep
fig = plt.figure(figsize=(20, 10))
rows, cols = 2, len(CAMERAS)

plot_idx = 1
for camera in CAMERAS:
  # rgb
  rgb_name = "%s_%s" % (camera, 'rgb')
  rgb = np.transpose(obs_dict[rgb_name], (1, 2, 0))
  fig.add_subplot(rows, cols, plot_idx)
  plt.imshow(rgb)
  plt.axis('off')
  plt.title("%s_rgb | step %s" % (camera, ts))

  # depth
  depth_name = "%s_%s" % (camera, 'depth')
  depth = np.transpose(obs_dict[depth_name], (1, 2, 0)).reshape(IMAGE_SIZE, IMAGE_SIZE)
  fig.add_subplot(rows, cols, plot_idx+len(CAMERAS))
  plt.imshow(depth)
  plt.axis('off')
  plt.title("%s_depth | step %s" % (camera, ts))

  plot_idx += 1

plt.show()